# Components

Components:
 - Convert file to Parquet (.to_parquet)
 - Export data from BQ as parquet to GCS
 - Fit the data (.fit)
 - Preprocess the data (.transform)

All the components will use FUSE as the main storage driver.  
Files will be read and write from/to GCS after each operation.

Components must be generic enough for customers to reuse them.

In [22]:
import kfp
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath)
from typing import Optional

from kfp import LocalClient, run_pipeline_func_locally

In [3]:
client = kfp.LocalClient()

/opt/conda/lib/python3.7/site-packages/kfp/_local_client.py:184: FutureWarning: LocalClient is an Alpha[1] feature. It may be deprecated in the future.
[1] https://github.com/kubeflow/pipelines/blob/master/docs/release/feature-stages.md#alpha
  category=FutureWarning,


In [17]:
BASE_IMAGE_NAME='us-east1-docker.pkg.dev/renatoleite-mldemos/docker-images/nvt-conda'

In [18]:
def test_comp():
    import logging
    import nvtabular as nvt
    import numpy as np
    import os
    
    from dask_cuda import LocalCUDACluster
    from dask.distributed import Client
    print('passou')

In [13]:
def test_pipeline():
    test = test_comp()

In [21]:
test = kfp.components.create_component_from_func_v2(test_comp, base_image=BASE_IMAGE_NAME, install_kfp_package=False)

In [23]:
run_result = run_pipeline_func_locally(
    test_pipeline,
    {},
    execution_mode=LocalClient.ExecutionMode("local"),
)

/opt/conda/lib/python3.7/site-packages/kfp/_local_client.py:184: FutureWarning: LocalClient is an Alpha[1] feature. It may be deprecated in the future.
[1] https://github.com/kubeflow/pipelines/blob/master/docs/release/feature-stages.md#alpha
  category=FutureWarning,


passou
